# Analyzing Ride-Sharing Patterns: Insights for Zuber
by Mikhail Karepov

Zuber, a ride-sharing company. To help Zuber succeed, this project analyzes data on taxi rides and weather in the city. The goal is to find patterns, understand passenger preferences, and explore how factors like weather affect rides.

Using SQL and Python, the analysis will uncover key insights to guide Zuber’s decisions and improve its services. From popular neighborhoods to the impact of weather, this project will provide a clear picture of what influences ride-sharing.

**Table of contents**<a id='toc0_'></a>    
- 1. [Initialization](#toc1_)    
- 2. [Load Data](#toc2_)    
- 3. [Prepare the Data](#toc3_)    
  - 3.1. [General Info](#toc3_1_)    
  - 3.2. [Fix Data](#toc3_2_)    
    - 3.2.1. [Data Types](#toc3_2_1_)    
    - 3.2.2. [Duplicates](#toc3_2_2_)    
    - 3.2.3. [Enriching Data](#toc3_2_3_)    
- 4. [Final Data](#toc4_)    
- 5. [Data Analysis](#toc5_)    
  - 5.1. [Neighborhood Drop-offs](#toc5_1_)    
  - 5.2. [Taxi Companies and Number of Rides](#toc5_2_)    
- 6. [Testing Hypotheses](#toc6_)    
- 7. [Overall Conclusions](#toc7_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=3
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

### 1. <a id='toc1_'></a>[Initialization](#toc0_)

In [1]:
# Importing all the necessary libraries
import pandas as pd  # For data manipulation and analysis
import plotly.express as px  # For interactive data visualization
import numpy as np  # For numerical operations
from scipy import stats as st  # For statistical tests and functions

### 2. <a id='toc2_'></a>[Load Data](#toc0_)

In [2]:
rides_data = pd.read_csv('./datasets/project_sql_result_01.csv')  # Taxi companies and rides
neighborhoods_data = pd.read_csv('./datasets/project_sql_result_04.csv')  # Drop-off locations and average trips
loop_ohare_data = pd.read_csv('./datasets/project_sql_result_07.csv') # Loop to O'Hare rides with weather

### 3. <a id='toc3_'></a>[Prepare the Data](#toc0_)

#### 3.1. <a id='toc3_1_'></a>[General Info](#toc0_)

The `rides_data` table (data on taxi companies and rides):

* **company_name** — taxi company name
* **trips_amount** — the number of rides for each taxi company

The `neighborhoods_data` table (data on drop-off neighborhoods):

* **dropoff_location_name** — Chicago neighborhoods where rides ended
* **average_trips** — the average number of rides that ended in each neighborhood

The `loop_ohare_data` table (data on Loop to O'Hare rides):

* **start_ts** — pickup date and time
* **weather_conditions** — weather conditions at the moment the ride started
* **duration_seconds** — ride duration in seconds

In [3]:
print(rides_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   company_name  64 non-null     object
 1   trips_amount  64 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB
None


In [4]:
print(neighborhoods_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   dropoff_location_name  94 non-null     object 
 1   average_trips          94 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.6+ KB
None


In [5]:
print(loop_ohare_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068 entries, 0 to 1067
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   start_ts            1068 non-null   object 
 1   weather_conditions  1068 non-null   object 
 2   duration_seconds    1068 non-null   float64
dtypes: float64(1), object(2)
memory usage: 25.2+ KB
None


In [6]:
display(rides_data.sample(10))
display(neighborhoods_data.sample(10))
display(loop_ohare_data.sample(10))

,company_name,trips_amount
46,3620 - 52292 David K. Cab Corp.,21
5,Chicago Carriage Cab Corp,9181
4,Taxi Affiliation Service Yellow,9299
11,Globe Taxi,4383
48,3623 - 72222 Arrington Enterprises,20
58,5062 - 34841 Sam Mestas,8
57,Metro Group,11
27,Service Taxi Association,402
33,Metro Jet Taxi A,146
24,Chicago Taxicab,1014


,dropoff_location_name,average_trips
40,Bucktown,74.300000
20,Rush & Division,395.533333
57,Hermosa,27.566667
30,United Center,178.900000
78,Gage Park,8.233333
4,O'Hare,2546.900000
65,Ashburn,16.133333
83,Avalon Park,6.966667
19,Old Town,469.566667
37,Bridgeport,77.100000


,start_ts,weather_conditions,duration_seconds
55,2017-11-25 14:00:00,Good,1620.0
155,2017-11-18 10:00:00,Bad,2055.0
809,2017-11-04 17:00:00,Bad,2472.0
607,2017-11-11 16:00:00,Good,2213.0
777,2017-11-11 09:00:00,Good,1494.0
151,2017-11-18 06:00:00,Good,1420.0
170,2017-11-04 12:00:00,Good,2220.0
618,2017-11-11 14:00:00,Good,2340.0
364,2017-11-11 10:00:00,Good,1709.0
499,2017-11-11 12:00:00,Good,2351.0


Data Inspection and Observations

Inspecting the `rides_data` dataset:

- **Data Types:**
  - **Object (`object`)**: `company_name`
  - **Integer (`int64`)**: `trips_amount`

- **Missing Values:**
  - No missing values in this dataset.

Inspecting the `neighborhoods_data` dataset:

- **Data Types:**
  - **Object (`object`)**: `dropoff_location_name`
  - **Float (`float64`)**: `average_trips`

- **Missing Values:**
  - No missing values in this dataset.

Inspecting the `loop_ohare_data` dataset:

- **Data Types:**
  - **Object (`object`)**: `start_ts`, `weather_conditions`
  - **Float (`float64`)**: `duration_seconds`

- **Missing Values:**
  - No missing values in this dataset.

Next Steps:

1. **Convert Data Types:**
   - Convert `start_ts` in `loop_ohare_data` to datetime format for easier analysis.

2. **Check for Duplicates:**
   - Ensure there are no duplicate entries in the datasets to maintain data integrity.

#### 3.2. <a id='toc3_2_'></a>[Fix Data](#toc0_)

##### 3.2.1. <a id='toc3_2_1_'></a>[Data Types](#toc0_)

In [7]:
# Convert `start_ts` in `loop_ohare_data` to datetime format
loop_ohare_data['start_ts'] = pd.to_datetime(loop_ohare_data['start_ts'])
print(loop_ohare_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068 entries, 0 to 1067
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   start_ts            1068 non-null   datetime64[ns]
 1   weather_conditions  1068 non-null   object        
 2   duration_seconds    1068 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 25.2+ KB
None


##### 3.2.2. <a id='toc3_2_2_'></a>[Duplicates](#toc0_)

In [8]:
# Check for duplicates in `rides_data`
print(f"Number of duplicate rows in rides_data: {rides_data.duplicated().sum()}")

# Check for duplicates in `neighborhoods_data`
print(f"Number of duplicate rows in neighborhoods_data: {neighborhoods_data.duplicated().sum()}")

# Check for duplicates in `loop_ohare_data`
print(f"Number of duplicate rows in loop_ohare_data: {loop_ohare_data.duplicated().sum()}")

Number of duplicate rows in rides_data: 0
Number of duplicate rows in neighborhoods_data: 0
Number of duplicate rows in loop_ohare_data: 197


Why Keep Duplicates in `loop_ohare_data`:
- The duplicates happen because the data is grouped by hour.
- It’s normal to have multiple rides starting in the same hour with the same weather and duration.
- Keeping these duplicates helps us understand how many trips happened during that time.

##### 3.2.3. <a id='toc3_2_3_'></a>[Enriching Data](#toc0_)

In [9]:
# Calculate total rides
total_rides = rides_data['trips_amount'].sum()

# Add market share column
rides_data['market_share'] = (rides_data['trips_amount'] / total_rides) * 100

# Display enriched data
display(rides_data.head())

,company_name,trips_amount,market_share
0,Flash Cab,19558,14.243578
1,Taxi Affiliation Services,11422,8.318343
2,Medallion Leasin,10367,7.550014
3,Yellow Cab,9888,7.201171
4,Taxi Affiliation Service Yellow,9299,6.772218


### 4. <a id='toc4_'></a>[Final Data](#toc0_)

In [10]:
print(rides_data.info())
print()
print(neighborhoods_data.info())
print()
print(loop_ohare_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   company_name  64 non-null     object 
 1   trips_amount  64 non-null     int64  
 2   market_share  64 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.6+ KB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   dropoff_location_name  94 non-null     object 
 1   average_trips          94 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.6+ KB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068 entries, 0 to 1067
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   start_ts            1068

In [11]:
display(rides_data.sample(10))
display(neighborhoods_data.sample(10))
display(loop_ohare_data.sample(10))

,company_name,trips_amount,market_share
58,5062 - 34841 Sam Mestas,8,0.005826
60,2733 - 74600 Benny Jona,7,0.005098
33,Metro Jet Taxi A,146,0.106328
9,Blue Ribbon Taxi Association Inc.,5953,4.335414
16,Blue Diamond,2070,1.507527
36,3591 - 63480 Chuks Cab,37,0.026946
1,Taxi Affiliation Services,11422,8.318343
5,Chicago Carriage Cab Corp,9181,6.686282
27,Service Taxi Association,402,0.292766
3,Yellow Cab,9888,7.201171


,dropoff_location_name,average_trips
64,Oakland,16.633333
89,Mount Greenwood,3.137931
57,Hermosa,27.566667
37,Bridgeport,77.100000
81,Morgan Park,7.700000
28,Wicker Park,182.600000
78,Gage Park,8.233333
14,Near South Side,839.833333
23,Rogers Park,299.033333
22,Lincoln Square,356.733333


,start_ts,weather_conditions,duration_seconds
960,2017-11-11 10:00:00,Good,1486.0
333,2017-11-11 06:00:00,Good,1260.0
116,2017-11-11 08:00:00,Good,1260.0
758,2017-11-25 15:00:00,Good,2195.0
1050,2017-11-11 16:00:00,Good,2280.0
153,2017-11-04 14:00:00,Good,3480.0
381,2017-11-18 12:00:00,Bad,2640.0
563,2017-11-11 06:00:00,Good,1200.0
269,2017-11-25 10:00:00,Good,1676.0
398,2017-11-11 17:00:00,Good,2640.0


### 5. <a id='toc5_'></a>[Data Analysis](#toc0_)

#### 5.1. <a id='toc5_1_'></a>[Neighborhood Drop-offs](#toc0_)

In this section, we look at which neighborhoods in Chicago have the most drop-offs. By focusing on the top 10 neighborhoods, we can see where passengers are most active and where ride-sharing demand is the highest.

**Steps**:
- Find the top 10 neighborhoods with the most average drop-offs.
- Create a bar chart to show the results.

**Goal**: 
To figure out which neighborhoods need more focus for ride-sharing services and understand where passengers go most often.

In [12]:
top_10_neighborhoods = neighborhoods_data.sort_values(by='average_trips', ascending=False).head(10)
max_value = top_10_neighborhoods['average_trips'].max()

# Create a bar chart
fig = px.bar(top_10_neighborhoods,
             x='dropoff_location_name',
             y='average_trips',
             title="Top 10 Neighborhoods by Drop-offs",
             labels={'dropoff_location_name': 'Neighborhood', 'average_trips': 'Average Drop-offs'},
             text='average_trips',
             template='ggplot2')

# Format text to show integer values instead of long decimals
fig.update_traces(texttemplate='%{text:.0f}', textposition='outside')

fig.update_layout(yaxis=dict(title='Average Drop-offs',
                             automargin=True,
                             range=[0, max_value * 1.2]),
                             xaxis_tickangle=-45)

# Display the plot
fig.show()

**What We Found**

The drop-offs are mostly concentrated in a few busy neighborhoods:
1. **Loop**, **River North**, and **Streeterville** have the most drop-offs. These areas likely include business districts, popular attractions, or high-density residential zones.
2. Other neighborhoods like **West Loop**, **O'Hare**, and **Lake View** are also popular but have fewer drop-offs compared to the top three.
3. Places like **Grant Park** and **Gold Coast** have moderate demand but are still important.

**Conclusion**

The **Loop** has the most drop-offs, making it a clear focus area for ride-sharing services. To improve services:
- Add more cars in the busiest neighborhoods, especially during peak hours.
- Look into when rides are most frequent in these areas to adjust availability.

#### 5.2. <a id='toc5_2_'></a>[Taxi Companies and Number of Rides](#toc0_)

In this section, we analyze which taxi companies had the highest number of rides. By focusing on the top-performing companies, we can identify key players in the market and understand ride distribution among them.

**Steps**:
- Calculate the total number of rides for each taxi company.
- Create a bar chart to visualize the top-performing companies.

**Goal**: 
To identify the companies with the most rides and understand their market dominance. This will help Zuber position itself in the competitive landscape.


In [13]:
# Sort and select the top 10 companies by number of rides
top_companies = rides_data.sort_values(by='trips_amount', ascending=False).head(10)
max_rides = top_companies['trips_amount'].max()

# Create a bar chart
fig = px.bar(top_companies,
             x='company_name',
             y='trips_amount',
             title="Top 10 Taxi Companies by Number of Rides",
             labels={'company_name': 'Taxi Company', 'trips_amount': 'Number of Rides'},
             text='trips_amount',
             template='ggplot2')

fig.update_traces(texttemplate='%{text:.0f}', textposition='outside')

fig.update_layout(yaxis=dict(title='Number of Rides',
                             automargin=True,
                             range=[0, max_rides * 1.2]),
                             xaxis_tickangle=-45)

# Show the plot
fig.show()

**What We Found**

1. **Flash Cab** is the most popular taxi company, with the highest number of rides. This shows it has a strong presence in the market.
2. Companies like **Taxi Affiliation Services**, **Medallion Leasing**, and **Yellow Cab** are also big players with many rides.
3. Some companies, like **City Service** and **Sun Taxi**, have a fair amount of rides but are less dominant than the top ones.
4. Smaller companies, like **Blue Ribbon Taxi Association Inc.**, are in the top 10 but have far fewer rides compared to the leaders.

**Conclusion**

- The Chicago taxi market is led by a few big companies, like **Flash Cab** and **Taxi Affiliation Services**, that get most of the rides.
- For **Zuber**, being a ride-sharing service:
  - Learn from what the top companies do well, like how they price rides and attract customers.
  - Compete with smaller taxi companies by offering faster pickups, better service, or cheaper rides.
  - Focus on busy neighborhoods and times where demand is high to attract more customers.

This analysis helps Zuber find ways to compete with taxis and grow in the Chicago market.

### 6. <a id='toc6_'></a>[Testing Hypotheses](#toc0_)

**Hypothesis**:  
*"The average duration of rides from the Loop to O'Hare International Airport changes on rainy Saturdays."*



**Formulating the Hypotheses:**

- **Null Hypothesis (H₀)**: The average duration of rides on rainy Saturdays is the same as the average duration on non-rainy Saturdays.
  
- **Alternative Hypothesis (H₁)**: The average duration of rides on rainy Saturdays is different from the average duration on non-rainy Saturdays.

**Significance Level (α)**:  
We’ll use α = 0.05, meaning there’s a 5% chance of rejecting the null hypothesis if it’s actually true.

**Why the t-test?**

We are comparing the means of two independent samples: ride durations on rainy Saturdays vs. non-rainy Saturdays.  
The **t-test** is appropriate for this because:
1. We’re testing the difference between two means.
2. It accounts for small sample sizes and assumes data is normally distributed (which we can check).

In [14]:
# Convert 'start_ts' to datetime
loop_ohare_data['start_ts'] = pd.to_datetime(loop_ohare_data['start_ts'])

# Filter data for Saturdays
loop_ohare_data['day_of_week'] = loop_ohare_data['start_ts'].dt.dayofweek  # Monday=0, Saturday=5
saturdays_data = loop_ohare_data[loop_ohare_data['day_of_week'] == 5]

# Separate data into rainy and non-rainy Saturdays
rainy_saturdays = saturdays_data[saturdays_data['weather_conditions'] == 'Bad']['duration_seconds']
non_rainy_saturdays = saturdays_data[saturdays_data['weather_conditions'] == 'Good']['duration_seconds']

# Perform a t-test
t_stat, p_value = st.ttest_ind(rainy_saturdays, non_rainy_saturdays, equal_var=False)

# Output results
print(f"T-statistic: {t_stat}")
print(f"P-value: {p_value}")

# Interpret results
alpha = 0.05
if p_value < alpha:
    print("We reject the null hypothesis: The average ride duration changes on rainy Saturdays.")
else:
    print("We fail to reject the null hypothesis: No significant difference in average ride duration.")

T-statistic: 7.186034288068629
P-value: 6.738994326108734e-12
We reject the null hypothesis: The average ride duration changes on rainy Saturdays.


In [15]:
# Create a box plot directly from the loop_ohare_data
fig = px.box(loop_ohare_data,
             x='duration_seconds',
             y='weather_conditions',
             title="Comparison of Ride Duration on Rainy and Non-Rainy Saturdays",
             labels={'duration_seconds': 'Duration (seconds)', 'weather_conditions': 'Weather Conditions'},
             template='ggplot2')

fig.update_layout(xaxis_title='Duration (seconds)',
                  yaxis_title='Weather Conditions',
                  xaxis=dict(tickangle=0),  # Ensure horizontal x-axis labels
                  yaxis=dict(automargin=True))

# Show the plot
fig.show()

Conclusion

The box plot shows that ride durations from the Loop to O'Hare are longer on rainy Saturdays compared to non-rainy Saturdays.


### 7. <a id='toc7_'></a>[Overall Conclusions](#toc0_)

1. **Popular Neighborhoods**: The Loop and River North are the top drop-off locations, indicating high demand in these areas. Zuber could focus on optimizing services in these neighborhoods.

2. **Taxi Companies**: A few companies dominate the market, such as Flash Cab and Taxi Affiliation Services. Understanding their strategies could help Zuber compete effectively.

3. **Impact of Weather**: Rainy weather significantly increases ride durations, especially on Saturdays. Zuber should prepare for potential delays and adjust pricing or resource allocation accordingly.

4. **Insights for Growth**: By targeting high-demand neighborhoods, monitoring competitor trends, and addressing weather-related challenges, Zuber can improve its operations and meet customer expectations effectively.